In [8]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install underthesea

  Obtaining dependency information for underthesea from https://files.pythonhosted.org/packages/c0/26/a2615fb23899ec747eaf7d7b3d4ae6b1959b71ae2a8546e121a8ce375d9c/underthesea-6.8.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
     -------- ------------------------------ 30.7/139.4 kB 1.4 MB/s eta 0:00:01
     ---------- -------------------------- 41.0/139.4 kB 667.8 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/139.4 kB 667.8 kB/s eta 0:00:01
     ---------------- -------------------- 61.4/139.4 kB 409.6 kB/s eta 0:00:01
     ------------------------------- ---- 122.9/139.4 kB 554.9 kB/s eta 0:00:01
     ---------------------------------- - 133.1/139.4 kB 563.7 kB/s eta 0:00:01
     ---------------------------------- - 133.1/139.4 kB 563.7 kB/s eta 0:00:01
     ------------------------------------ 139.4/139.4 kB 414.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - --

In [4]:
!pip install findspark


In [37]:
import csv
from pymongo import MongoClient

# Kết nối đến MongoDB
client = MongoClient('mongodb://localhost:27017')  # Thay thế URL kết nối của bạn nếu cần

# Chọn cơ sở dữ liệu và bộ sưu tập
db = client['youtube_db']  # Thay thế 'your_database_name' bằng tên cơ sở dữ liệu của bạn
collection = db['commentvideo']

# Lấy dữ liệu từ MongoDB
data = collection.find({}, {"author": 1, "comment": 1, "video_id": 1, "playlist_program": 1, "_id": 0})

# Tạo tệp CSV và ghi dữ liệu vào đó

with open('../data/output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["author", "comment", "video_id", "playlist_program"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=',')
    
    writer.writeheader()
    
    for item in data:
        writer.writerow(item)


In [32]:
import findspark
findspark.init()
import pandas as pd
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = (
    SparkSession.builder.appName("MySparkApp")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

pd.options.display.max_columns = None
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 150
sns.set(color_codes=True)

In [68]:
import re
import sys

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace

# Khởi tạo Spark Session
spark = SparkSession.builder.appName("YourAppNameHere").getOrCreate()

schema = StructType([
    StructField("author", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("video_id", StringType(), True),
    StructField("playlist_program", StringType(), True)
])

# Tạo một DataFrame từ một tập dữ liệu JSON và chỉ định schema và 'multiline' option
IN_PATH_RAW = "../data/commentvideo_export.json"
df_train_raw = spark.read.schema(schema).option('multiline', True).json(IN_PATH_RAW)

# Xử lý làm sạch cột "comment" bằng cách loại bỏ emoticon và thẻ HTML
df_cleaned = df_train_raw.withColumn("comment_cleaned", regexp_replace(df_train_raw["comment"],  "[^\x00-\x7F]+", ""))
# df_cleaned = remove_emoji(df_train_raw)
# Hiển thị DataFrame sau khi làm sạch
df_cleaned.show(truncate=False)

# Đừng quên ngắt kết nối Spark khi bạn đã hoàn thành công việc.
# spark.stop()


+-----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|author           |comment                                                                                                                                                       

In [ ]:
from underthesea import sentiment
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a UDF to analyze sentiment
def analyze_sentiment(comment):
   sentiment_result = sentiment(comment)
   return sentiment_result

# Create a UDF for sentiment analysis
sentiment_udf = udf(analyze_sentiment, StringType())

# Apply the sentiment analysis UDF to the DataFrame
df_with_sentiment = df_cleaned.withColumn("sentiment", sentiment_udf(df_cleaned["comment_cleaned"]))
df_with_sentiment.select("comment_cleaned", "sentiment").show(truncate=False)

In [72]:

spark.stop()

root
 |-- author;comment;video_id;playlist_program: string (nullable = true)

